In [1]:
import random
import time
import datetime
import pendulum as pdt
import pandas as pd
import pandas_ta as ta
#getting all active orders
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import OrderSide, QueryOrderStatus
import pandas as pd
api_key='PKB872RM09BHVAMWPS9P'
secret_key='yFTUvVwM5f9G0mrZJWFIZkZu5eF4mxbAq5b4wAFy'
trading_client = TradingClient(api_key, secret_key, paper=True)


from alpaca.data.historical import CryptoHistoricalDataClient
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()
from zoneinfo import ZoneInfo
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

import logging
logging.basicConfig(level=logging.INFO, filename=f'open_range_{datetime.date.today()}',filemode='w',format="%(asctime)s - %(message)s")

try:
    order_filled_dataframe=pd.read_csv('order_filled_list.csv')
    order_filled_dataframe.set_index('time',inplace=True)

except:
    column_names = ['time','ticker','price','action']
    order_filled_dataframe = pd.DataFrame(columns=column_names)
    order_filled_dataframe.set_index('time',inplace=True)


tickers=["BTC/USD","ETH/USD"]

In [7]:
!pip3 install numpy==1.26.4

  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


In [1]:
!pip3 install pendulum

  Using cached pendulum-3.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.9 kB)
  Using cached time_machine-2.15.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (21 kB)
Using cached pendulum-3.0.0-cp312-cp312-macosx_11_0_arm64.whl (352 kB)
Using cached time_machine-2.15.0-cp312-cp312-macosx_11_0_arm64.whl (17 kB)


In [2]:
def trade_buy_stocks(ticker,closing_price):
    print('placing market order')
    # preparing orders
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=1,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.GTC
                        )

    # Market order
    market_order = trading_client.submit_order(
                    order_data=market_order_data
                )
    a=[ticker,closing_price,'BUY']
    order_filled_dataframe.loc[pdt.now('UTC')] = a
    order_filled_dataframe.to_csv('order_filled_list.csv')
    

In [3]:
from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
    CryptoLatestQuoteRequest
    )

def get_current_price_crypto(symbol):
    crypto_historical_data_client = CryptoHistoricalDataClient()
    req = CryptoLatestQuoteRequest(
        symbol_or_symbols = [symbol],
    )
    res = crypto_historical_data_client.get_crypto_latest_quote(req)
    cp=(res[symbol].ask_price+res[symbol].bid_price)/2
    print(cp)
    return cp

In [6]:
p=get_current_price_crypto(tickers[1])
trade_buy_stocks(tickers[1],p)

2643.7025000000003
placing market order


In [7]:

def close_this_position(ticker_name):
    try:
        position = trading_client.get_open_position(ticker_name.replace('/',''))
        print(position)
        c=trading_client.close_position(ticker_name.replace('/',''))
        print(c)
        print('position closed')
        p=get_current_price_crypto(ticker_name)
        a=[ticker_name,p,c.side]
        order_filled_dataframe.loc[pdt.now('UTC')] = a
        order_filled_dataframe.to_csv('order_filled_list.csv')
    except:
        print('position does not exist')
close_this_position(tickers[1])

asset_id=UUID('35f33a69-f5d6-4dc9-b158-4485e5e92e4b') symbol='ETHUSD' exchange=<AssetExchange.CRYPTO: 'CRYPTO'> asset_class=<AssetClass.CRYPTO: 'crypto'> asset_marginable=False avg_entry_price='2644.2' qty='0.9978' side=<PositionSide.LONG: 'long'> market_value='2637.724212' cost_basis='2638.38276' unrealized_pl='-0.658548' unrealized_plpc='-0.0002496029044702' unrealized_intraday_pl='-0.658548' unrealized_intraday_plpc='-0.0002496029044702' current_price='2643.54' lastday_price='2669.9' change_today='-0.0098730289523952' swap_rate=None avg_entry_swap_rate=None usd=None qty_available='0.9978'
id=UUID('f4e562ed-a244-4edb-8c25-96843b8105bd') client_order_id='aa7f16c1-78ad-4152-b838-191acb61a4bb' created_at=datetime.datetime(2024, 9, 29, 10, 37, 42, 112362, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 9, 29, 10, 37, 42, 113158, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 9, 29, 10, 37, 42, 112362, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None fa

In [22]:
close_this_position(tickers[1].replace('/',''))

asset_id=UUID('35f33a69-f5d6-4dc9-b158-4485e5e92e4b') symbol='ETHUSD' exchange=<AssetExchange.CRYPTO: 'CRYPTO'> asset_class=<AssetClass.CRYPTO: 'crypto'> asset_marginable=False avg_entry_price='2644.5' qty='0.9978' side=<PositionSide.LONG: 'long'> market_value='2635.010196' cost_basis='2638.6821' unrealized_pl='-3.671904' unrealized_plpc='-0.0013915674040461' unrealized_intraday_pl='-3.671904' unrealized_intraday_plpc='-0.0013915674040461' current_price='2640.82' lastday_price='2669.9' change_today='-0.0108917937001386' swap_rate=None avg_entry_swap_rate=None usd=None qty_available='0.9978'
id=UUID('ff146005-8610-4fe0-9e45-eedd8c8ebac4') client_order_id='8d3688f8-acd0-4d95-8645-bc25ce6d8259' created_at=datetime.datetime(2024, 9, 29, 10, 33, 30, 681250, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 9, 29, 10, 33, 30, 682593, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 9, 29, 10, 33, 30, 681250, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None fai

In [27]:
ticker_name=tickers[1]
position = trading_client.get_open_position(ticker_name.replace('/',''))
print(position)
c=trading_client.close_position(ticker_name.replace('/',''))
print(c)
print('position closed')
p=get_current_price_crypto(ticker_name)
a=[ticker_name,p,c.side]
order_filled_dataframe.loc[pdt.now('UTC')] = a
order_filled_dataframe.to_csv('order_filled_list.csv')

asset_id=UUID('35f33a69-f5d6-4dc9-b158-4485e5e92e4b') symbol='ETHUSD' exchange=<AssetExchange.CRYPTO: 'CRYPTO'> asset_class=<AssetClass.CRYPTO: 'crypto'> asset_marginable=False avg_entry_price='2643.56' qty='0.9978' side=<PositionSide.LONG: 'long'> market_value='2635.057093' cost_basis='2637.744168' unrealized_pl='-2.687075' unrealized_plpc='-0.001018701901647' unrealized_intraday_pl='-2.687075' unrealized_intraday_plpc='-0.001018701901647' current_price='2640.867' lastday_price='2669.9' change_today='-0.010874190044571' swap_rate=None avg_entry_swap_rate=None usd=None qty_available='0.9978'
id=UUID('64980c8a-1ab6-4a2c-94b4-21db10201e18') client_order_id='ad0ed7cd-5b19-4bc7-81fa-feb75d6d53d1' created_at=datetime.datetime(2024, 9, 29, 10, 35, 16, 707579, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 9, 29, 10, 35, 16, 708766, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 9, 29, 10, 35, 16, 707579, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None fa

In [19]:
c.side

<OrderSide.SELL: 'sell'>